In [5]:
# Delay Line Interface

In [111]:
import sys
import numpy as np
import time
import matplotlib.pyplot as plt
sys.path.append("C:/Users/alber/Downloads")  # PATH to hardware_ops.py

In [112]:
from hardware_ops import HardwareOps

In [113]:
from hardware_ops import HardwareOps

config_filename =  r'C:\Users\alber\Documents\hardware_project\hardware_config.json' # use this if you ned to store serial port numbers
ops = HardwareOps(config_filename) # initialized inside the __init__ functions

In [ ]:
# Call piezo serials:
piezo_list = np.array(ops.piezo_serials_list)
y_piezo_list = piezo_list[[1,3]]
x_piezo_list = piezo_list[[0,2]]

# Zero Piezo Controllers to 75V (Our 'zero' value)
ops.piezos.set_to_voltage_zero_value(piezo_list, 75)

29253216 set to 0.0503555406353954
29253216 set to 1.0503555406353955
29253216 set to 2.0503555406353957
29253216 set to 3.0503555406353957
29253216 set to 4.050355540635396
29253216 set to 5.050355540635396
29253216 set to 6.050355540635396
29253216 set to 7.050355540635396
29253216 set to 8.050355540635396
29253216 set to 9.050355540635396
29253216 set to 10.050355540635396
29253216 set to 11.050355540635396
29253216 set to 12.050355540635396
29253216 set to 13.050355540635396
29253216 set to 14.050355540635396
29253216 set to 15.050355540635396
29253216 set to 16.050355540635394
29253216 set to 17.050355540635394
29253216 set to 18.050355540635394
29253216 set to 19.050355540635394
29253216 set to 20.050355540635394
29253216 set to 21.050355540635394
29253216 set to 22.050355540635394
29253216 set to 23.050355540635394
29253216 set to 24.050355540635394
29253216 set to 25.050355540635394
29253216 set to 26.050355540635394
29253216 set to 27.050355540635394
29253216 set to 28.0503555

Sensitivity Matrix (Not relevant at the moment)

In [225]:
delta = 49 # Voltage change

def compute_sensitivity_piezo_matrix(x0, y0, delta): # Sensitivity Matrix for only the piezo actuators
    """ Computes the sensitivity matrix for piezo actuators
    inputs: 
        x0 : initial voltage in actuators, or vector 'a'
        y0 : initial position in quad cell, or vector 'q'

    outputs:
        M : sensitivity matrix for relationship between actuator voltage and quad cell laser position

    """
    n_inputs = 4  # Number of piezo actuators
    n_outputs = len(y0)

    M = np.zeros((n_outputs, n_inputs))  # Sensitivity Matrix Initialization

    piezo_serials = ops.piezo_serials_list  # Serial Numbers

    for j in range(n_inputs):
        x0_perturb = x0.copy()

        # Perturb piezo actuator j by delta
        ops.piezos.set_voltage(piezo_serials[j], x0_perturb[j] + delta)
        time.sleep(0.3)

        y_perturb = ops.quads.get_xy_position_tavg()  # Read qc values after perturbed voltage change [x1, y1, x2, y2]
        time.sleep(0.3)

        # Reset actuator j to initial value
        ops.piezos.set_voltage(piezo_serials[j], x0_perturb[j])
        time.sleep(0.3)

        # Compute and store the finite difference
        M[:, j] = (np.array(y_perturb) - np.array(y0)) / delta

    return M

In [ ]:
delta = 10 # Voltage change

# Sensitivity Matrix for only the X or Y piezo actuators
def compute_sensitivity_piezo_matrix(x0, y0, delta): # Sensitivity Matrix for only the piezo actuators
    """ Computes the sensitivity matrix for piezo actuators
    inputs: 
        x0 : initial voltage in actuators, or vector 'a'
        y0 : initial position in quad cell, or vector 'q'

    outputs:
        M : sensitivity matrix for relationship between actuator voltage and quad cell laser position

    """
    n_inputs = 2  # Number of piezo actuators
    n_outputs = len(y0)

    M = np.zeros((n_outputs, n_inputs))  # Sensitivity Matrix Initialization

    y_piezos = np.array(ops.piezo_serials_list)
    y_piezos = y_piezos[[1,3]]

    for j in range(n_inputs):
        x0_perturb = x0.copy()

        # Perturb piezo actuator j by delta
        ops.piezos.set_voltage(y_piezos[j], x0_perturb[j] + delta)
        time.sleep(0.3)

        y_perturb = np.array(ops.quads.get_xy_position_tavg())[[1, 3]]  # Read qc values after perturbed voltage change [x1, y1, x2, y2]
        time.sleep(0.3)

        # Reset actuator j to initial value
        ops.piezos.set_voltage(y_piezos[j], x0_perturb[j])
        time.sleep(0.3)

        # Compute and store the finite difference
        M[:, j] = (np.array(y_perturb) - np.array(y0)) / delta

    return M

In [265]:
x0 = np.array(ops.piezos.get_all_voltages())[[1,3]]
print(x0)
y0 = np.array(ops.quads.get_xy_position_tavg())[[1,3]]
print(y0)

[50.13885922 50.09308145]
[-0.02223703 -0.05548873]


In [267]:
M = compute_sensitivity_piezo_matrix(x0, y0, delta)

29253237 set to 50.1358073671682
29253237 set to 51.1358073671682
29253237 set to 52.1358073671682
29253237 set to 53.1358073671682
29253237 set to 54.1358073671682
29253237 set to 55.1358073671682
29253237 set to 56.1358073671682
29253237 set to 57.1358073671682
29253237 set to 58.1358073671682
29253237 set to 59.1358073671682
29253237 set to 60.1358073671682
29253237 set to 60.1388592181158
29253237 set to 60.1916562395093
29253237 set to 59.1916562395093
29253237 set to 58.1916562395093
29253237 set to 57.1916562395093
29253237 set to 56.1916562395093
29253237 set to 55.1916562395093
29253237 set to 54.1916562395093
29253237 set to 53.1916562395093
29253237 set to 52.1916562395093
29253237 set to 51.1916562395093
29253237 set to 50.1916562395093
29253237 set to 50.1388592181158
29253246 set to 50.0900296029542
29253246 set to 51.0900296029542
29253246 set to 52.0900296029542
29253246 set to 53.0900296029542
29253246 set to 54.0900296029542
29253246 set to 55.0900296029542
29253246 s

In [269]:
print(M)
q = ops.quads.get_xy_position_tavg()
q = np.array(q)[[1, 3]]
print(q)

[[-0.00154661 -0.001393  ]
 [-0.00230038 -0.00215334]]
[-0.0230138  -0.05724882]


In [277]:
x_hat = np.linalg.solve(M, -q)  # or lstsq(M, q)
print(x_hat)

[ 239.71090565 -282.66553036]


In [ ]:
# Checking the current voltage and applying the solution
current_volt = np.array(ops.piezos.get_all_voltages())[[0,2]]
print(current_volt)

for i in range(len(x_piezo_list)):
    ops.piezos.set_voltage(x_piezo_list[i], current_volt[i]+x_hat[i])

[50.12970367 50.00305185]
29253216 set to 50.129703665273
29253216 set to 49.129703665273
29253216 set to 48.129703665273
29253216 set to 47.129703665273
29253216 set to 46.129703665273
29253216 set to 45.129703665273
29253216 set to 44.129703665273
29253216 set to 43.129703665273
29253216 set to 42.129703665273
29253216 set to 41.129703665273
29253216 set to 40.129703665273
29253216 set to 39.129703665273
29253216 set to 38.129703665273
29253216 set to 37.129703665273
29253216 set to 36.129703665273
29253216 set to 35.129703665273
29253216 set to 34.129703665273
29253216 set to 33.129703665273
29253216 set to 32.129703665273
29253216 set to 31.129703665273
29253216 set to 30.129703665273
29253216 set to 29.129703665273
29253216 set to 28.129703665273
29253216 set to 27.129703665273
29253216 set to 26.129703665273
29253216 set to 25.129703665273
29253216 set to 24.129703665273
29253216 set to 23.129703665273
29253216 set to 22.129703665273
29253216 set to 21.129703665273
29253216 set t

Centering Quadcells Code

In [ ]:
# Simple code that ensures the beam position in the quadcells is relatively centered before running shift_slope
def easy_center(final_limit=0.02):
    stage_ser = ops.stages.serial_numbers[0]
    step = 1  # Step size for voltage adjustment

    # Get initial QC values
    qc1 = ops.quads.get_xy_position(sig_strength=0.045)[0]
    qc2 = ops.quads.get_xy_position(sig_strength=0.045)[2]

    # Determine direction to move
    if qc1 > 0 and qc2 > 0:
        direction = -1
    elif qc1 < 0 and qc2 < 0:
        direction = 1
    else:
        print(f"Already straddling zero: QC1={qc1:.3f}, QC2={qc2:.3f}")
        return

    print(f"Starting easy_center with direction {direction}, initial QC1={qc1:.3f}, QC2={qc2:.3f}")

    current_voltage = ops.stages.get_position(stage_ser, 'chan4')

    while True:
        current_voltage += direction * step
        ops.stages.move_absolute(stage_ser, 'chan4', current_voltage)
        time.sleep(0.3)

        qc1 = ops.quads.get_xy_position(sig_strength=0.045)[0]
        qc2 = ops.quads.get_xy_position(sig_strength=0.045)[2]

        print(f"Moved to {current_voltage:.2f} | QC1={qc1:.3f}, QC2={qc2:.3f}")

        # ✅ Stop if at least one value flipped sign from its original side
        if direction == 1 and (qc1 > 0 or qc2 > 0):
            print("At least one QC flipped from negative to positive. Stopping.")
            break
        elif direction == -1 and (qc1 < 0 or qc2 < 0):
            print("At least one QC flipped from positive to negative. Stopping.")
            break

        # Also allow early exit if centered well enough
        if abs(qc1) + abs(qc2) <= final_limit:
            print("Close enough to center. Stopping.")
            break


In [ ]:
# Adjusts "slope" of the exiting beam from the delay line. Clockwise movement on M1/M2 decreases slope, counter-clockwise movement increases slope.
def shift_slope(current_volt_chan2, current_volt_chan4, qc1_threshold=0.09, qc2_threshold=0.11, final_limit=0.015,
                min_moves_chan2=4, min_moves_chan4=2, sig_strength = 0.045):
    easy_center()
    step = 1
    stage_ser = ops.stages.serial_numbers[0]

    # Get quadcell values after centering
    qc1 = ops.quads.get_xy_position(sig_strength)[0]
    qc2 = ops.quads.get_xy_position(sig_strength)[2]

    # Early exit if too far apart
    if abs(qc1) > qc1_threshold and abs(qc2) > qc2_threshold:
        print(f"Too far apart after centering: QC1={qc1:.3f}, QC2={qc2:.3f} — aborting.")
        return

    # Determine walk order
    start_with_chan4 = (abs(qc1) > qc1_threshold or abs(qc2) > qc2_threshold)
    direction = 1 if qc1 > qc2 else -1

    print("Starting shift_slope...")
    print(f"Initial QC1={qc1:.3f}, QC2={qc2:.3f}, starting with {'chan4' if start_with_chan4 else 'chan2'}")

    def chan2_loop():
        nonlocal current_volt_chan2
        while True:
            qc1, qc2 = ops.quads.get_xy_position(sig_strength)[0], ops.quads.get_xy_position(sig_strength)[2]
            if abs(qc1) + abs(qc2) <= final_limit:
                print(f"Exit: QC1={qc1:.3f}, QC2={qc2:.3f}")
                return True

            # Stop chan2 based on threshold-crossing in the direction
            if (direction == 1 and (qc1 < -qc1_threshold or qc2 < -qc2_threshold)) or \
                (direction == -1 and (qc1 > qc1_threshold or qc2 > qc2_threshold)):
                print(f"Threshold crossed on chan2: QC1={qc1:.3f}, QC2={qc2:.3f}")
                return False

            current_volt_chan2 += direction * step
            ops.stages.move_absolute(stage_ser, 'chan2', current_volt_chan2)
            time.sleep(0.3)
            qc1, qc2 = ops.quads.get_xy_position(sig_strength)[0], ops.quads.get_xy_position(sig_strength)[2]
            print(f"[chan2] Moved to {current_volt_chan2}, QC: {qc1:.3f}, {qc2:.3f}")

        # --- chan4 loop (SLOW axis, fewer moves needed) ---
    def chan4_loop():
        nonlocal current_volt_chan4
        while True:
            qc1, qc2 = ops.quads.get_xy_position(sig_strength)[0], ops.quads.get_xy_position(sig_strength)[2]
            if abs(qc1) + abs(qc2) <= final_limit:
                print(f"Exit: QC1={qc1:.3f}, QC2={qc2:.3f}")
                return True

            # Stop chan4 based on threshold-crossing in the direction
            if (direction == 1 and (qc1 > qc1_threshold or qc2 > qc2_threshold)) or \
                (direction == -1 and (qc1 < -qc1_threshold or qc2 < -qc2_threshold)):
                print(f"Threshold crossed on chan4: QC1={qc1:.3f}, QC2={qc2:.3f}")
                return False

            current_volt_chan4 += direction * step
            ops.stages.move_absolute(stage_ser, 'chan4', current_volt_chan4)
            time.sleep(0.3)
            qc1, qc2 = ops.quads.get_xy_position(sig_strength)[0], ops.quads.get_xy_position(sig_strength)[2]
            print(f"[chan4] Moved to {current_volt_chan4}, QC: {qc1:.3f}, {qc2:.3f}")
            
    # Alternate between chan2 and chan4 until final_limit is reached
    while True:
        qc1, qc2 = ops.quads.get_xy_position(sig_strength)[0], ops.quads.get_xy_position(sig_strength)[2]
        if abs(qc1) + abs(qc2) <= final_limit:
            print(f"Final exit: QC1={qc1:.3f}, QC2={qc2:.3f}")
            return

        if start_with_chan4:
            chan4_loop()
            chan2_loop()
        else:
            chan2_loop()
            chan4_loop()

In [ ]:
piezo_ser = ops.piezo_serials_list

# Adjusts the vertical piezo voltage on both M1 and M2 to vertical center beam on quadcells
def piezo_center_vertical():
    step = 0.5  # Voltage increment step
    max_voltage = 150  # Maximum voltage
    min_voltage = 0    # Minimum voltage (assuming piezos can't go negative)

    while True:
        # Get current quadcell Y positions
        qc1, qc2 = ops.quads.get_xy_position()[1], ops.quads.get_xy_position()[3]
        total = qc1 + qc2

        if abs(total) < 0.01:
            break  # Centered enough

        # Get current voltages on the piezos
        p1, p2 = ops.piezos.get_all_voltages()[1], ops.piezos.get_all_voltages()[3]

        # Determine direction: subtract if too negative, add if too positive
        direction = -1 if total < 0 else 1

        # Apply bounded voltage adjustment
        new_p1 = max(min(p1 + direction * step, max_voltage), min_voltage)
        new_p2 = max(min(p2 + direction * step, max_voltage), min_voltage)

        ops.piezos.set_voltage(piezo_ser[1], new_p1)
        ops.piezos.set_voltage(piezo_ser[3], new_p2)

        time.sleep(0.3)


In [ ]:
# Performs a similar method to shift_slope to further center the beams in the quadcells using only the piezo actuators.
def piezo_center_horizontal(current_p1, current_p2, qc_threshold=0.02):
    piezo_ser = ops.piezo_serials_list
    step = 1
    min_voltage = 0
    max_voltage = 150

    qc1, qc2 = ops.quads.get_xy_position()[0], ops.quads.get_xy_position()[2]
    direction = 1 if qc1 > qc2 else -1

    print("Starting piezo_center_horizontal...")

    def p1_loop():
        nonlocal current_p1
        while True:
            qc1, qc2 = ops.quads.get_xy_position()[0], ops.quads.get_xy_position()[2]
            if abs(qc1) + abs(qc2) <= 0.007:
                print(f"Exit: QC1={qc1:.3f}, QC2={qc2:.3f}")
                return True

            if (direction == 1 and (qc1 < -qc_threshold or qc2 < -qc_threshold)) or \
               (direction == -1 and (qc1 > qc_threshold or qc2 > qc_threshold)):
                print(f"Threshold crossed on piezo1: QC1={qc1:.3f}, QC2={qc2:.3f}")
                return False

            current_p1 = max(min(current_p1 + direction * step, max_voltage), min_voltage)
            ops.piezos.set_voltage(piezo_ser[0], current_p1)
            time.sleep(0.3)
            print(f"[piezo1] Moved to {current_p1:.2f}, QC: {qc1:.3f}, {qc2:.3f}")

    def p2_loop():
        nonlocal current_p2
        while True:
            qc1, qc2 = ops.quads.get_xy_position()[0], ops.quads.get_xy_position()[2]
            if abs(qc1) + abs(qc2) <= 0.007:
                print(f"Exit: QC1={qc1:.3f}, QC2={qc2:.3f}")
                return True

            if (direction == 1 and (qc1 > qc_threshold or qc2 > qc_threshold)) or \
               (direction == -1 and (qc1 < -qc_threshold or qc2 < -qc_threshold)):
                print(f"Threshold crossed on piezo2: QC1={qc1:.3f}, QC2={qc2:.3f}")
                return False

            current_p2 = max(min(current_p2 + direction * step, max_voltage), min_voltage)
            ops.piezos.set_voltage(piezo_ser[2], current_p2)
            time.sleep(0.3)
            print(f"[piezo2] Moved to {current_p2:.2f}, QC: {qc1:.3f}, {qc2:.3f}")

    while True:
        qc1, qc2 = ops.quads.get_xy_position()[0], ops.quads.get_xy_position()[2]
        if abs(qc1) + abs(qc2) <= 0.007:
            print(f"Final Exit: QC1={qc1:.3f}, QC2={qc2:.3f}")
            return

        p1_loop()
        p2_loop()


In [140]:
print(ops.stages.get_all_positions())
print(ops.piezos.get_all_voltages())

[63500.0, 97.0, 0.0, -106.0]
[92.1048615985595, 68.8222907193213, 94.9980162968841, 69.0328684347056]


In [ ]:
# Adjust any of the stages if desired.
stage_ser = ops.stage_serials[0]
ops.stages.move_absolute(stage_ser, 'chan4', -65)

Moving to position -65
Move Complete


In [ ]:
# Identifies the quadcell positions and current voltage values on the stages. Necessary for running shift_slope.
qc_positions = ops.quads.get_xy_position(sig_strength=0.05)
qc1pos, qc2pos = qc_positions[0], qc_positions[2]
print(qc1pos, qc2pos)
current_volt_chan2, current_volt_chan4 = ops.stages.get_all_positions()[1], ops.stages.get_all_positions()[3]
print(current_volt_chan2, current_volt_chan4)

0.074 0.019
-86.0 -177.0


In [ ]:
# Sets the piezos voltage all to 75, so that we have the option of precise piezo centering afterwards.
# Must run the preceding code cell before running this
ops.piezos.set_to_voltage_zero_value(x_piezo_list, 75)
shift_slope(current_volt_chan2, current_volt_chan4, sig_strength=0.045)

29253216 set to 75.0755333109531
29253216 set to 75.0
113250437 set to 74.9977111117893
113250437 set to 75.0
Starting easy_center with direction -1, initial QC1=0.074, QC2=0.019
Moving to position -178
Move Complete
Moved to -178.00 | QC1=0.020, QC2=-0.088
At least one QC flipped from positive to negative. Stopping.
Starting shift_slope...
Initial QC1=0.020, QC2=-0.088, starting with chan2
Moving to position -85
Move Complete
[chan2] Moved to -85.0, QC: 0.008, -0.101
Moving to position -84
Move Complete
[chan2] Moved to -84.0, QC: -0.005, -0.111
Threshold crossed on chan2: QC1=-0.005, QC2=-0.111
Moving to position -176
Move Complete
[chan4] Moved to -176.0, QC: 0.062, -0.012
Moving to position -175
Move Complete
[chan4] Moved to -175.0, QC: 0.082, 0.047
Moving to position -174
Move Complete
[chan4] Moved to -174.0, QC: 0.094, 0.093
Threshold crossed on chan4: QC1=0.094, QC2=0.093
Moving to position -83
Move Complete
[chan2] Moved to -83.0, QC: 0.089, 0.071
Moving to position -82
Move 

In [ ]:
# Centers quadcells vertically
piezo_center_vertical()

In [135]:
ops.quads.get_xy_position()
current_p1, current_p2 = ops.piezos.get_all_voltages()[0], ops.piezos.get_all_voltages()[2]
piezo_center_horizontal(current_p1, current_p2)

Starting piezo_center_horizontal...
29253216 set to 75.0709555345317
29253216 set to 76.0709555345317
[piezo1] Moved to 76.07, QC: 0.007, -0.000
29253216 set to 76.1329996642964
29253216 set to 77.0709555345317
[piezo1] Moved to 77.07, QC: 0.006, -0.002
29253216 set to 77.1446882534257
29253216 set to 78.0709555345317
[piezo1] Moved to 78.07, QC: 0.005, -0.003
29253216 set to 78.1701101718192
29253216 set to 79.0709555345317
[piezo1] Moved to 79.07, QC: 0.004, -0.006
29253216 set to 79.1543321024201
29253216 set to 80.0709555345317
[piezo1] Moved to 80.07, QC: 0.003, -0.009
29253216 set to 80.1705984679708
29253216 set to 81.0709555345317
[piezo1] Moved to 81.07, QC: 0.002, -0.011
29253216 set to 81.1456648457289
29253216 set to 82.0709555345317
[piezo1] Moved to 82.07, QC: 0.000, -0.015
29253216 set to 82.1527756584368
29253216 set to 83.0709555345317
[piezo1] Moved to 83.07, QC: -0.001, -0.017
29253216 set to 83.1644642475661
29253216 set to 84.0709555345317
[piezo1] Moved to 84.07, 

Shutdown

In [148]:
ops.piezos.shutdown()
ops.quads.shutdown()
ops.stages.shutdown()

[INFO] Setting 29253216 to 0 V/position before shutdown.
29253216 set to 92.1094393749809
29253216 set to 91.1094393749809
29253216 set to 90.1094393749809
29253216 set to 89.1094393749809
29253216 set to 88.1094393749809
29253216 set to 87.1094393749809
29253216 set to 86.1094393749809
29253216 set to 85.1094393749809
29253216 set to 84.1094393749809
29253216 set to 83.1094393749809
29253216 set to 82.1094393749809
29253216 set to 81.1094393749809
29253216 set to 80.1094393749809
29253216 set to 79.1094393749809
29253216 set to 78.1094393749809
29253216 set to 77.1094393749809
29253216 set to 76.1094393749809
29253216 set to 75.1094393749809
29253216 set to 74.1094393749809
29253216 set to 73.1094393749809
29253216 set to 72.1094393749809
29253216 set to 71.1094393749809
29253216 set to 70.1094393749809
29253216 set to 69.1094393749809
29253216 set to 68.1094393749809
29253216 set to 67.1094393749809
29253216 set to 66.1094393749809
29253216 set to 65.1094393749809
29253216 set to 64.